In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from textblob import TextBlob

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten,BatchNormalization
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

In [ ]:
data = pd.read_csv("training_set_rel3.tsv", sep='\t', encoding = "ISO-8859-1")

In [ ]:
data.head(2)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
col_to_keep =['essay','domain1_score']
filter_data = data[col_to_keep]


In [ ]:
def word_counting(x):
    return (len(TextBlob(x).words))
filter_data['word_length'] = filter_data['essay'].apply(word_counting)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def sentence_counting(x):
    sentence_len = len([len(sentence.split(' ')) for sentence in TextBlob(x).sentences])
    return sentence_len
filter_data['no_of_sentence'] = filter_data['essay'].apply(sentence_counting)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
def avg_sentence_sentiment(x):
    sentiment_essay = TextBlob(x).sentiment.polarity
    return sentiment_essay
filter_data['sentiment_essay'] = filter_data['essay'].apply(avg_sentence_sentiment)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
def avg_length_of_words(x):
    word_len = [len(word) for word in TextBlob(x).words]
    return ( sum(word_len)/len(word_len) )
filter_data['avg_word_len']  = filter_data['essay'].apply(avg_length_of_words)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
def grammar_check(x):
    tool = language_check.LanguageTool('en-US')
    matches = tool.check(x)
    return len(matches)
filter_data['Grammar_check'] = filter_data['essay'].apply(grammar_check)

NameError: ignored

In [ ]:
display(filter_data)

,essay,domain1_score,word_length,no_of_sentence,sentiment_essay,avg_word_len
0,"Dear local newspaper, I think effects computer...",8,343,16,0.310471,4.358601
1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,422,20,0.274000,4.331754
2,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,283,14,0.340393,4.356890
3,"Dear Local Newspaper, @CAPS1 I have found that...",10,527,27,0.266828,4.851992
4,"Dear @LOCATION1, I know having computers has a...",8,470,30,0.199684,4.378723
...,...,...,...,...,...,...
12971,In most stories mothers and daughters are eit...,35,863,27,0.146336,4.018540
12972,I never understood the meaning laughter is th...,32,556,35,0.147192,3.881295
12973,"When you laugh, is @CAPS5 out of habit, or is ...",40,827,41,0.131967,4.355502
12974,Trippin' on fen...,40,576,39,-0.001249,4.133681


In [ ]:
y=data['domain1_score']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(filter_data, y, test_size=0.3, random_state=42)

In [ ]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [ ]:
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    BatchNormalization()
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu', kernel_initializer='he_normal'))
    model.compile(loss='mean_squared_error', optimizer='RMSProp', metrics=['mae'])
    model.summary()
    return model

In [ ]:
#Training Word2Vec model
num_features = 300
min_word_count = 20
num_workers = -1
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 size=num_features, 
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index2word)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model[i])        
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[] 

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [ ]:
!pip install language_check

  ERROR: Failed building wheel for language-check
  Running setup.py clean for language-check
Failed to build language-check
    Running setup.py install for language-check ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-sk_tq3fr/language-check/setup.py'"'"'; __file__='"'"'/tmp/pip-install-sk_tq3fr/language-check/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-648let1y/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [ ]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:

training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 1, 300)            721200    
_________________________________________________________________
lstm_16 (LSTM)               (None, 64)                93440     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=150)

Epoch 1/150
9083/9083 [==============================] - 2s 246us/step - loss: 11.7338 - mae: 1.8896
Epoch 2/150
9083/9083 [==============================] - 2s 235us/step - loss: 12.0019 - mae: 1.9238
Epoch 3/150
9083/9083 [==============================] - 2s 241us/step - loss: 11.6604 - mae: 1.9148
Epoch 4/150
9083/9083 [==============================] - 2s 242us/step - loss: 11.2400 - mae: 1.8913
Epoch 5/150
9083/9083 [==============================] - 2s 238us/step - loss: 11.5344 - mae: 1.8933
Epoch 6/150
9083/9083 [==============================] - 2s 243us/step - loss: 11.5241 - mae: 1.8827
Epoch 7/150
9083/9083 [==============================] - 2s 242us/step - loss: 11.6067 - mae: 1.8913
Epoch 8/150
9083/9083 [==============================] - 2s 244us/step - loss: 11.7342 - mae: 1.9164
Epoch 9/150
9083/9083 [==============================] - 2s 243us/step - loss: 10.8501 - mae: 1.8375
Epoch 10/150
9083/9083 [==============================] - 2s 241us/step - loss: 11.9491 - m

In [ ]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred 

array([[2.],
       [1.],
       [9.],
       ...,
       [2.],
       [8.],
       [3.]], dtype=float32)

In [ ]:
lstm_model.evaluate(testing_vectors,y_test)

3893/3893 [==============================] - 0s 96us/step


[nan, nan]

In [ ]:
!pip install adaptnlp

     |████████████████████████████████| 81kB 2.4MB/s 
     |████████████████████████████████| 7.6MB 5.3MB/s 
     |████████████████████████████████| 753.4MB 18kB/s 
     |████████████████████████████████| 143kB 26.6MB/s 
     |████████████████████████████████| 573kB 52.1MB/s 
     |████████████████████████████████| 983kB 46.4MB/s 
     |████████████████████████████████| 256kB 52.9MB/s 
     |████████████████████████████████| 798kB 56.3MB/s 
     |████████████████████████████████| 3.7MB 51.5MB/s 
     |████████████████████████████████| 890kB 54.9MB/s 
     |████████████████████████████████| 1.1MB 53.9MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=173495686a997a77e42a5aa5791652954f42c9b29a793b8887bab98251d2d9b7
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for sqlitedict: filename=sqlitedict-1.6.0-cp36-non

In [ ]:
from adaptnlp import EasyWordEmbeddings, EasyStackedEmbeddings, EasyDocumentEmbeddings
import torch

In [ ]:
# Instantiate with variable number of language models
embeddings = EasyDocumentEmbeddings("bert-base-cased", "xlnet-base-cased")

May need a couple moments to instantiate...
Pooled embedding loaded
RNN embeddings loaded


In [ ]:
example_text=filter_data['essay'].tolist()
def semantic(i):
  sentences = embeddings.embed_pool(example_text[i])
  # Get the text/document embedding
  for sentence in sentences:
    a=sentence.get_embedding()
    a=a.detach().numpy()
    mean=a.mean()
    #display(a)
    #dff=pd.DataFrame({'semantic':mean}, index=(i for i in range(1)))
    #isplay(dff)
    
for i in range(1):
  semantic(i)
  k=[]
  k.append(mean)


In [ ]:
display(k)

[-0.015247249]

In [ ]:
import keras
keras.models.load_model("final_lstm.h5")